In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()

In [4]:
pdf = pd.read_excel('/Users/gabriele.sabato/Downloads/Import_POs_old.xls', sheet_name='Tabelle1')


In [5]:
pdf

,PI,Freight_Service,purchase_document_number,SAM,supplier_name,vendor,location_code,PI_Date,Departure_Port,EK_Price_LCY_tot,Sum_CBM
0,2020.51,Rhenus,4800005579,Thejaswara Polu,INDEX LIVING GmbH - Hauptsitz,100252,9001,2020-12-19,INMUN,886.05,1.830
1,2020.51,Rhenus,4800005579,Thejaswara Polu,INDEX LIVING GmbH - Hauptsitz,100252,9001,2020-12-19,INMUN,1181.40,2.340
2,2020.51,Rhenus,4800005579,Thejaswara Polu,INDEX LIVING GmbH - Hauptsitz,100252,9001,2020-12-19,INMUN,1265.60,2.040
3,2020.51,Rhenus,4800005579,Thejaswara Polu,INDEX LIVING GmbH - Hauptsitz,100252,9001,2020-12-19,INMUN,1319.50,2.590
4,2020.51,Rhenus,4800005579,Thejaswara Polu,INDEX LIVING GmbH - Hauptsitz,100252,9001,2020-12-19,INMUN,1672.11,8.181
...,...,...,...,...,...,...,...,...,...,...,...
9389,2021.27,Forto,4800009662,Denis Wu,Actona Company AS (Inland),100001,9001,2021-07-09,CNNGB,4430.72,12.212
9390,2021.27,Forto,4800009662,Denis Wu,Actona Company AS (Inland),100001,9001,2021-07-09,CNNGB,6026.00,18.700
9391,2021.28,Zencargo,4800009387,Thejaswara Polu,"Ningbo Frrede Furniture Co.,Ltd.",100640,9001,2021-07-15,CNNGB,3752.93,19.451
9392,2021.28,Zencargo,4800009387,Thejaswara Polu,"Ningbo Frrede Furniture Co.,Ltd.",100640,9001,2021-07-15,CNNGB,8922.06,48.384


In [6]:
df = spark.createDataFrame(pdf)
df.createOrReplaceTempView('Cont_cost')

In [7]:
df_list_of_docks = spark.sql(""" SELECT DISTINCT Departure_Port from Cont_cost where where PI <= '2021.06' """)

list_of_docks = df_list_of_docks.rdd.map(lambda x: x.Departure_Port).collect()

print(list_of_docks)

['Ho Chi Minh City', 'Tianjin', 'CNXAM', 'CNSGH', 'CNHUA', 'CNXIP', 'CNYTN', 'VNSGN', 'CNSZX', 'Ningbo', 'CNNGB', 'IDSRG', 'MYTPP', 'Semarang', 'BRITJ', 'MYPGU', 'INMUN', 'CNTSN', 'IDSUB', 'VNUIH', 'Shenzhen', 'IDJKT', 'CNQIN', 'VNDAD', 'Shanghai']


In [10]:
import re
# Function to match the string
def match(lista):
        # regex
        pattern = '[A-Z]+[a-z]+$'
        list_of_cities = []
        # searching pattern
        for city in lista:
            if re.search(pattern, city):
                list_of_cities.append(city)
        return list_of_cities
list_of_city_low = match(list_of_docks)

print(list_of_city_low)
list_of_city_low.remove('Semarang')

['Ho Chi Minh City', 'Tianjin', 'Ningbo', 'Semarang', 'Shenzhen', 'Shanghai']


In [37]:
df_filt= spark.sql("""SELECT PI,
                        Freight_service,
                        Departure_Port,
                        EK_Price_LCY_tot,
                        purchase_document_number,
                        Sum_CBM,
                       CASE WHEN country_code in ('ID','IN','MY','VN')
                       THEN 'SEA'
                        ELSE country_code END as area_code
                        FROM (SELECT PI,
                        Freight_service,
                        Departure_Port,
                        EK_Price_LCY_tot,
                        purchase_document_number,
                        Sum_CBM,
                        CASE WHEN Departure_port in ('Ho Chi Minh City', 'Tianjin', 'Ningbo', 'Semarang', 'Shenzhen', 'Shanghai')
                        THEN 'CN'
                        WHEN Departure_port == 'Semarang'
                        THEN 'ID'
                        ELSE substring(Departure_port,1,2) END as country_code
                        from Cont_cost where PI <= '2021.06') """)

df_filt.createOrReplaceTempView('Cont_filt')


In [38]:
df_filt.show(10000,False)


+-------+------------------------------+----------------+------------------+------------------------+-------------------+---------+
|PI     |Freight_service               |Departure_Port  |EK_Price_LCY_tot  |purchase_document_number|Sum_CBM            |area_code|
+-------+------------------------------+----------------+------------------+------------------------+-------------------+---------+
|2020.51|Rhenus                        |INMUN           |886.05            |4800005579              |1.83               |SEA      |
|2020.51|Rhenus                        |INMUN           |1181.4            |4800005579              |2.3400000000000003 |SEA      |
|2020.51|Rhenus                        |INMUN           |1265.6            |4800005579              |2.04               |SEA      |
|2020.51|Rhenus                        |INMUN           |1319.5            |4800005579              |2.59               |SEA      |
|2020.51|Rhenus                        |INMUN           |1672.11           |

In [50]:
df_group_by = spark.sql("""SELECT PI,
                         Freight_service,
                         area_code,
                         COUNT(DISTINCT purchase_document_number) as N_of_containers,
                         SUM(EK_Price_LCY_tot) as Tot_EK_Price_LCY,
                         SUM(Sum_CBM) as TOT_CBM
                    FROM Cont_filt
                    WHERE area_code != 'BR'
                    GROUP BY PI, Freight_service, area_code
                    ORDER BY PI,area_code,Freight_service""")

df_group_by.show(1000,False)



+-------+------------------------------+---------+---------------+------------------+------------------+
|PI     |Freight_service               |area_code|N_of_containers|Tot_EK_Price_LCY  |TOT_CBM           |
+-------+------------------------------+---------+---------------+------------------+------------------+
|2020.51|Forto                         |CN       |63             |1173286.3499999996|4147.961000000001 |
|2020.51|Zencargo                      |CN       |12             |190955.66         |733.7600000000001 |
|2020.51|Forto                         |SEA      |22             |484643.02         |1429.6349999999998|
|2020.51|Rhenus                        |SEA      |40             |901695.5899999997 |2340.5359999999996|
|2020.51|Zencargo                      |SEA      |6              |171051.5          |375.741           |
|2020.52|Forto                         |CN       |35             |667022.4599999998 |2271.2            |
|2020.52|Zencargo                      |CN       |16   

In [51]:
df_group_by.createOrReplaceTempView('Cont_areas')


In [54]:
df_per_week = spark.sql("""SELECT PI,
                            SUM(N_of_containers),
                            SUM(Tot_EK_Price_LCY),
                            SUM(TOT_CBM)
                            FROM Cont_areas
                            GROUP BY PI
                            ORDER BY PI""")

df_per_week.show(1000,False)

+-------+--------------------+---------------------+------------------+
|PI     |sum(N_of_containers)|sum(Tot_EK_Price_LCY)|sum(TOT_CBM)      |
+-------+--------------------+---------------------+------------------+
|2020.51|143                 |2921632.119999999    |9027.633000000002 |
|2020.52|93                  |1916375.55           |5896.4619999999995|
|2020.53|103                 |2098669.35           |6645.638999999998 |
|2021.01|100                 |1754935.4699999997   |6269.192          |
|2021.02|135                 |2682467.2399999998   |8799.777          |
|2021.03|75                  |1479395.42           |4822.244999999999 |
|2021.04|92                  |1730949.71           |5720.803000000001 |
|2021.05|57                  |1409762.2700000003   |3584.592          |
|2021.06|18                  |339370.35000000003   |1173.9009999999998|
+-------+--------------------+---------------------+------------------+



In [43]:
df_group_by.toPandas().to_excel('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/container_CN_SEA.xlsx')

In [55]:
df_per_week.toPandas().to_excel('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Tot_Price_POs.xlsx')